<h2>                 Predictive Measures for Mitigation of COVID-19 Community Transmission </h2>
<h3>                         Applied Data Science Capstone Project </h3>


<h3> 1. Introduction </h3>

<p> Some Texutal Stuff </p>

<h3> 2. Business Problem</h3>

<p> Textual Stuff </p>

<h3> 3. Data </h3>

<p>Since, we're taking the context of India, we'll be looking at Data-sets containing coordinates of Indian States. We will be exploring the latest patient dataset. We'll also be looking at a data-set which will contain the records of Individual Patient updated daily.</p>
<p>If more Data Sets are needed</p>

<h3> 4. Analysis </h3>

<p> Let's start the coding part by importing libraries </p>

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

<p> Let's import our data sets </p>

In [28]:
raw_data_url = 'https://api.covid19india.org/csv/latest/raw_data.csv'
raw_data_df = pd.read_csv(state_data_url)
raw_data_df.drop(['Source_1','Source_2','Source_3'], 1, inplace = True)

death_recovered_url = 'https://api.covid19india.org/csv/latest/death_and_recovered.csv'
death_recovered_df = pd.read_csv(death_recovered_url)

state_wise_url = 'https://api.covid19india.org/csv/latest/state_wise.csv'
state_wise_df = pd.read_csv(state_wise_url)
state_wise_df.drop('State_Notes',1,inplace = True)

state_wise_daily_url = 'https://api.covid19india.org/csv/latest/state_wise_daily.csv'
state_wise_daily_df = pd.read_csv(state_wise_daily_url)

# sources_list_url = 'https://api.covid19india.org/csv/latest/sources_list.csv'
# sources_list_df = pd.read_csv(sources_list_url)

statewise_tested_numbers_data_url = 'https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv'
statewise_tested_numbers_data_df = pd.read_csv(statewise_tested_numbers_data_url)

case_time_series_url = 'https://api.covid19india.org/csv/latest/case_time_series.csv'
case_time_series_df = pd.read_csv(case_time_series_url)

tested_numbers_icmr_data_url = 'https://api.covid19india.org/csv/latest/tested_numbers_icmr_data.csv'
tested_numbers_icmr_data_df = pd.read_csv(tested_numbers_icmr_data_url)

print('Datasets imported !')

Datasets imported !


<p> Let's clean our datasets as we use them </p>

In [33]:
state_wise_df = state_wise_df.iloc(1:)
state_wise_df

SyntaxError: invalid syntax (<ipython-input-33-92782de85b29>, line 1)